In [5]:
import math
import numpy as np
import torch
import pkbar
from unet3d.config import *
from utils.Visualization import ImageSliceViewer3D
from tqdm import tqdm
from torch.nn import CrossEntropyLoss
import glob
from torch.optim import Adam
from unet3d.dataset import SAIADDataset
from patchify import patchify
import nrrd

%load_ext autoreload
%autoreload 2
%matplotlib inline
excl_patients = ['SAIAD 1','SAIAD 2','SAIAD 5','SAIAD 7','SAIAD 9','SAIAD 11','SAIAD 12','SAIAD 13','SAIAD 14',
                    'SAIAD 15','SAIAD 20 vasculaire', 'SAIAD 20 BIS vasculaire', 'SAIAD 19 BIS vasculaire']


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
dataset = SAIADDataset(excl_patients=excl_patients, epochs=1000, n_batches=100)

Generating patients probabilities...
13
0
1
2
3
4
5
6
7
8
9
10
11
12


In [17]:
ImageSliceViewer3D(np.array(dataset.patients_probabilities[3]))

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [18]:
arr = np.zeros(dataset.patients_probabilities[0].shape)
for center in dataset.patients_centers[0]:
    arr[center[0], center[1], center[2]]+=1
ImageSliceViewer3D(arr)


interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [13]:
probs = dataset.patients_probabilities[0]
torch.multinomial(probs, 2)


RuntimeError: prob_dist must be 1 or 2 dim